In [1]:
import pandas as pd
import numpy as np

x_train = pd.read_csv('Recipes.csv')
y_train = pd.read_csv('Defects.csv')
x_test = pd.read_csv('recipe_test_fixed.csv')
defects_info = pd.read_csv('Defect_correlation_with_raw_materials.csv', sep=';')

y_train.drop('PIECE_ID', axis=1, inplace=True)

In [2]:
# GET    false_codes
CODES = defects_info['DEFECT_CODE'].tolist()

false_codes = []
for code in CODES:
    row_in_df = defects_info.loc[defects_info['DEFECT_CODE'] == code ]
    if row_in_df['FROM_RAW_MATERIAL'].item() == 'FALSE':
        false_codes.append(code) 

C:\Users\dosto\Anaconda3\envs\cmc_mu_lbl\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: `item` has been deprecated and will be removed in a future version
  import sys


In [3]:
# CALCULATE  HEATID_values
HEATID_values = []
HEATID_values = x_train['HEATID'].tolist()
HEATID_values = set(HEATID_values)
HEATID_values = sorted(HEATID_values)
print(len(HEATID_values))

12746


In [4]:
# DELETE HEAT ID with all FALSE codes
all_false_codes = 0
for value in HEATID_values:     
    rowsDf = y_train.loc[y_train['HEAT_ID'] == value]
    y = set(rowsDf['DEFECT_TYPE'].values.tolist())
    if len(y) == 0:  # no defects
        y={1}
    y = list(y)

    ok = False
    for nn in range(0,len(y)):
        if y[nn] not in false_codes:
            ok=True
    if ok == False:
        x_train = x_train[x_train.HEATID != value]
        y_train = y_train[y_train.HEAT_ID != value]
        all_false_codes += 1

print(all_false_codes)

3708


In [5]:
for CODE in false_codes:
    y_train = y_train[y_train.DEFECT_TYPE != CODE]

In [6]:
# RECALCULATE  HEATID_values
HEATID_values = []
HEATID_values = x_train['HEATID'].tolist()
HEATID_values = set(HEATID_values)
HEATID_values = sorted(HEATID_values)
print(len(HEATID_values))

9038


In [7]:
# x_train  redefinition in some cell
# ALL HEATID   ID sorted + 

In [8]:
x1_train=[]
x2_train=[]
labels=[]

for value in HEATID_values:  
    x1=[]
    x2=[]

    rowsDf = x_train.loc[x_train['HEATID'] == value ]
    material = set( rowsDf['Material_Code'].values.tolist() )
    material = list(material)#unique

    matAndWeight = []
    for single in material:
        materialDf = rowsDf.loc[x_train['Material_Code'] == single ]
        weight = sum(materialDf['WEIGHT'].values.tolist() )
        x1.append(single)
        x2.append(weight)

    rowsDf = y_train.loc[y_train['HEAT_ID'] == value]
    y = set( rowsDf['DEFECT_TYPE'].values.tolist() )
    if( len(y)==0 ):#no defects
        y={1}
    y = list(y)

    x1_train.append(x1)
    x2_train.append(x2)
    labels.append(y)

In [9]:
print(x1_train[1])
print(x2_train[1])
print(labels[1])

[113, 114, 115]
[50306.6, 3473.6, 2215.8]
[59, 63, 71]


In [10]:
# test heatid
HEATID_values = []
HEATID_values = x_test['HEATID'].tolist()
HEATID_values = set(HEATID_values)
HEATID_values = sorted(HEATID_values)

In [11]:
x1_test=[]
x2_test=[]

for value in HEATID_values:  
    x1=[]
    x2=[]

    rowsDf = x_test.loc[x_test['HEATID'] == value ]
    material = set( rowsDf['Material_Code'].values.tolist() )
    material = list(material)#unique

    matAndWeight = []
    for single in material:
        materialDf = rowsDf.loc[x_test['Material_Code'] == single ]
        weight = sum(materialDf['WEIGHT'].values.tolist() )
        x1.append(single)
        x2.append(weight)

    x1_test.append(x1)
    x2_test.append(x2)

In [12]:
all_x2 = [x2_train, x2_test]

In [13]:
length = []
for a in x2_train:
    length.append(len(a))     
    
print(min(length))
print(max(length))

length = []
for a in x2_test:
    length.append(len(a))     
    
print(min(length))
print(max(length))

1
14
2
14


In [14]:
from keras.preprocessing.sequence import pad_sequences
TR_x1 = pad_sequences(x1_train, maxlen=14)
TR_x2 = pad_sequences(x2_train, maxlen=14, dtype='float')
TE_x1 = pad_sequences(x1_test, maxlen=14)
TE_x2 = pad_sequences(x2_test, maxlen=14, dtype='float')

Using TensorFlow backend.


In [15]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

for_scale = np.concatenate((TR_x2, TE_x2), axis=0)
transformer = MinMaxScaler().fit_transform(for_scale)

weightTR = transformer[0:len(x1_train)]
weightTE = transformer[len(x1_train):len(transformer)]

In [16]:
dlen = []
for w in labels:
    dlen.append(len(w))
    
print(max(dlen))

17


In [17]:
len(set(y_train['DEFECT_TYPE'].values.tolist() ))+1 # because of 1

123

In [18]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from sklearn.preprocessing import MultiLabelBinarizer

test = pd.Series(labels)

mlb = MultiLabelBinarizer()

label_onehoted = pd.DataFrame(mlb.fit_transform(test),
                   columns=mlb.classes_,
                   index=test.index)
#label_onehoted.to_csv('123.csv',index=False)
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [19]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  ALL MATERIAL CODES
test = pd.Series(x1_train + x1_test)
mlb = MultiLabelBinarizer()
res1 = pd.DataFrame(mlb.fit_transform(test),
                   columns=mlb.classes_,
                   index=test.index)
#res.to_csv('tran.csv',index=False)
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [20]:
test = pd.Series(x1_train + x1_test)
mlb = MultiLabelBinarizer()
res2 = pd.DataFrame(mlb.fit_transform(test),
                   columns=mlb.classes_,
                   index=test.index)

In [21]:
allX2 = x2_train+x2_test
flat_list = []
for sublist in allX2:
    for item in sublist:
        flat_list.append(item)
        
flat_np = np.asarray(flat_list, dtype=np.float64)

In [22]:
z = res2.to_numpy(dtype=np.float64)

for rows in range(len(z)):
    for columns in range(len(z[0])):
        if z[rows][columns] == 1:
            z[rows][columns] = flat_np[0]
            flat_np = np.delete(flat_np, 0)

In [23]:
#  FINAL VALUES
X1_TRAIN = res1.to_numpy()[0:len(x1_train)]
X1_TEST = res1.to_numpy()[len(x1_train):len(transformer)]
X2_TRAIN = z[0:len(x1_train)]
X2_TEST = z[len(x1_train):len(transformer)]

LABELS = label_onehoted.to_numpy()

In [24]:
from scipy.stats import uniform, randint
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV
# from sklearn.metrics import auc, accuracy_score, confusion_matrix#, mean_squared_error

def display_scores(scores):
    print("Scores: {0}\nMean: {1:.3f}\nStd: {2:.3f}".format(scores, np.mean(scores), np.std(scores)))


def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


from sklearn.metrics import f1_score, make_scorer
f1_scorer = make_scorer(f1_score, average="micro",greater_is_better=True)

In [45]:
import xgboost as xgb
from sklearn.multiclass import OneVsRestClassifier

xgb_model = OneVsRestClassifier(xgb.XGBClassifier())

params = {
    "estimator__objective": ["binary:logistic"],
    "estimator__booster":['gbtree'],
    "estimator__min_child_weight":[ 1, 3, 5, 7 ],
    "estimator__colsample_bytree": uniform(0.7, 0.3),
    "estimator__gamma": uniform(0, 0.5),
    "estimator__learning_rate": uniform(0.03, 0.3),  
    "estimator__max_depth": randint(2, 15),
    "estimator__n_estimators": randint(100, 150), 
    "estimator__subsample": uniform(0.6, 0.4)
}

search = RandomizedSearchCV(xgb_model, param_distributions=params, 
                            n_iter=140, verbose=1,
                            return_train_score=True,
                            scoring=f1_scorer, n_jobs=4 )
search.fit(X2_TRAIN, LABELS)
#report_best_scores(search.cv_results_, 1)

print (search.best_params_)
print (search.best_score_)

Fitting 3 folds for each of 140 candidates, totalling 420 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 18.0min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed: 76.0min
[Parallel(n_jobs=4)]: Done 420 out of 420 | elapsed: 170.3min finished


{'estimator__booster': 'gbtree', 'estimator__colsample_bytree': 0.8095632951174561, 'estimator__gamma': 0.2520958161210098, 'estimator__learning_rate': 0.3017490596517869, 'estimator__max_depth': 13, 'estimator__min_child_weight': 7, 'estimator__n_estimators': 141, 'estimator__objective': 'binary:logistic', 'estimator__subsample': 0.6052065454109774}
0.08336043738372087


In [188]:
from sklearn.model_selection import train_test_split 
X, X_val, Y, Y_val = train_test_split(X2_TRAIN, LABELS, test_size=0.2,random_state=52 , shuffle = True) #random_state=52

from sklearn.tree import DecisionTreeClassifier
classif = OneVsRestClassifier(DecisionTreeClassifier())
classif.fit(X, Y)
predicted_values = classif.predict(X_val) 
print(f1_score(Y_val, predicted_values , average="micro"))  # LABELS

C:\Users\dosto\Anaconda3\envs\cmc_mu_lbl\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 5 is present in all training examples.
  str(classes[c]))
C:\Users\dosto\Anaconda3\envs\cmc_mu_lbl\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 95 is present in all training examples.
  str(classes[c]))


0.25176304654442877


In [ ]:
#               CROSS VALIDATION SCIKIT-LEARN

from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier,AdaBoostClassifier,GradientBoostingClassifier,RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, RidgeClassifier,SGDClassifier
from sklearn.model_selection import KFold

seed = 7
numpy.random.seed(seed)

X = X2_TRAIN
Y = LABELS

kfold = KFold(n_splits=10,shuffle=True,random_state=seed)

cvscores = []
for train, test in kfold.split(X, Y):
    #   ONLY FOR VOTING
    #m1 = OneVsRestClassifier( KNeighborsClassifier(n_neighbors=3)  )
    #m2 = OneVsRestClassifier( ExtraTreesClassifier(n_estimators = 50)  )
    m3 = OneVsRestClassifier( RandomForestClassifier(n_estimators=50, random_state= 101) ) 
    #m4 = OneVsRestClassifier( DecisionTreeClassifier() ) 
    #m5 = OneVsRestClassifier( SGDClassifier(loss="hinge", penalty="l2", max_iter=250)  ) 
    
    #from sklearn.ensemble import VotingClassifier
    #clf = [m1, m2, m3, m4]
    #eclf = VotingClassifier(estimators=[('m1', m1), ('m2', m2), ('m3', m3), ('m4', m4)], voting='soft')

    classif = OneVsRestClassifier( RandomForestClassifier(n_estimators=50, random_state= 101) ) 
    classif.fit(X[train], Y[train])

    y_pred = classif.predict(X[test])   
    print(y_pred[0])
    #y_pred1 = classif.predict(X[test])
    #y_pred = np.zeros(y_pred1.shape)
    #y_pred[y_pred1>0.3] = 1 
    
    scores = f1_score(Y[test], y_pred , average="micro")
    print("%s: %.3f%%" % ('f1 micro-average: ', scores))
    cvscores.append(scores)

print("%.3f%% (+/- %.3f%%)" % (np.mean(cvscores), np.std(cvscores)))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC

parameters = {
       'estimator__kernel': ['linear','poly','rbf','sigmoid','precomputed'],  
       'estimator__gamma': ['auto','scale'],  
       'estimator__shrinking': [True,False],  
    #   'estimator__cache_size': [1,2,8,32,64],  
       'estimator__decision_function_shape': ['ovr'],   
}

f1_scorer = make_scorer(f1_score, average="micro",greater_is_better=True)

clf = RandomizedSearchCV( OneVsRestClassifier(SVC(verbose=1)),
                          parameters, scoring=f1_scorer,n_jobs=4,verbose=1,n_iter = 1)

clf.fit(X2_TRAIN, LABELS)
print (clf.best_params_)
print (clf.best_score_)

In [ ]:
#m1 = OneVsRestClassifier( KNeighborsClassifier(n_neighbors=3)  )
#m2 = OneVsRestClassifier( ExtraTreesClassifier(n_estimators = 50)  )
#m3 = OneVsRestClassifier( RandomForestClassifier(n_estimators=50) ) 
#m4 = OneVsRestClassifier( Perceptron() )
#m5 = OneVsRestClassifier( DecisionTreeClassifier() ) 
#m6 = OneVsRestClassifier( SGDClassifier(loss="hinge", penalty="l2", max_iter=250) )
m7 = OneVsRestClassifier( MLPClassifier() )  #??? 
SVM
Linear
from sklearn.linear_model import LogisticRegression

In [119]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier 
from sklearn.model_selection import train_test_split

X, X_val, Y, Y_val = train_test_split(X2_TRAIN, LABELS, test_size=0.2,random_state=52 , shuffle = True)

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
import xgboost as xgb
import keras
from keras.models import Model
from keras.layers import Input, Dense,BatchNormalization

input1 = Input(shape=(29,) )
dense1 = Dense(400, activation="elu")(input1)
bn1 = BatchNormalization()(dense1)
dense2 = Dense(400, activation="elu")(bn1)
bn2 = BatchNormalization()(dense2)
dense3 = Dense(400, activation="elu")(bn2)
bn3 = BatchNormalization()(dense3)
output = Dense(123, activation="sigmoid")(bn3)
model_C =  Model(inputs=input1, outputs=output)

model_C.compile(loss='binary_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])

model_C.fit(X, Y, batch_size=64, epochs=24,verbose=0 )

y_pred1 = model_C.predict(X_val)  
y_pred = np.zeros(y_pred1.shape)
y_pred[y_pred1>0.25] = 1 
scores = f1_score(Y_val, y_pred , average="micro")
print(scores)

#mod1=xgb.XGBClassifier(booster= 'gbtree', colsample_bytree= 0.809,  gamma= 0.252,
#                      learning_rate=0.301,  max_depth= 15, min_child_weight= 7, n_estimators = 141,  
#                      objective = 'binary:logistic', subsample= 0.605, n_jobs=4)



#clf1 = OneVsRestClassifier( KNeighborsClassifier(weights= 'uniform', p=1, n_neighbors=3, leaf_size=10, algorithm  = 'ball_tree') )
#clf2 = OneVsRestClassifier(  mod1  )
#lf3 = OneVsRestClassifier(  mod2  )


#clf_test1.fit(X, Y)
#y_pred1 = clf_test1.predict(X_val)  
#y_pred = np.zeros(y_pred1.shape)
#y_pred[y_pred1>0.3] = 1 
#scores = f1_score(Y_val, y_pred , average="micro")
#print(scores)

#y_pred1 = clf_test2.predict(X_val)  
#y_pred = np.zeros(y_pred1.shape)
#y_pred[y_pred1>0.3] = 1 
#scores = f1_score(Y_val, y_pred , average="micro")
#print(scores)


#0.35413802584547704 35
#0.36283429831816927 30

#0.36424024356490453 25
#0.36843544857768046 24
#0.3660701704945752 24


#0.3652656821910892
#0.3693716611549224
#0.36969539182504557
#0.37216455209534793

0.3729081565423639


In [124]:
y_pred1 = model_C.predict(X_val)  
y_pred = np.zeros(y_pred1.shape)
y_pred[y_pred1>0.23] = 1 
scores = f1_score(Y_val, y_pred, average="micro")
print(scores)

#0.22   0.37333333333333335

#elu
#0.23 0.3742323599448552

0.3742323599448552


In [52]:
import warnings
warnings.filterwarnings('ignore')

import keras
from keras.models import Model
from keras.layers import Input, Dense,BatchNormalization
from sklearn.linear_model import SGDClassifier,Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
X, X_val, Y, Y_val = train_test_split(X2_TRAIN, LABELS, test_size=0.2, random_state=52, shuffle = True)

#Xtr, X_meta, Ytr, Y_meta = train_test_split(  X, Y, test_size=0.2,random_state=52 , shuffle = True)
iteration = 0
#(7230, 123)
pred_values1 = []
pred_values2 = []
pred_values3 = []
pred_values4 = []
pred_values5 = []
pred_values6 = []
pred_values7 = []
pred_values8 = []
GT_values = []

kfold = KFold(n_splits=10,shuffle=True,random_state=101)

for train, test in kfold.split(X, Y):
    iteration += 1
    print(iteration)
        
    clf1 = OneVsRestClassifier( KNeighborsClassifier(weights= 'uniform', p=1, n_neighbors=3, leaf_size=10, algorithm  = 'ball_tree') )  
    clf2 = OneVsRestClassifier( ExtraTreesClassifier(n_estimators=75) ) 
    clf3 = OneVsRestClassifier( RandomForestClassifier(n_estimators= 5, max_features= None, criterion= 'entropy', class_weight=None)  )
    clf7 = OneVsRestClassifier( xgb.XGBClassifier(booster= 'gbtree', colsample_bytree= 0.809,  gamma= 0.252,learning_rate=0.301, max_depth= 15,
                                                  min_child_weight= 7, n_estimators = 141,  objective = 'binary:logistic', subsample= 0.605, n_jobs=4) )
    
    input1 = Input(shape=(29,) )
    dense1 = Dense(400, activation="relu")(input1)
    bn1 = BatchNormalization()(dense1)
    dense2 = Dense(400, activation="relu")(bn1)
    bn2 = BatchNormalization()(dense2)# 
    dense3 = Dense(400, activation="relu")(bn2)
    bn3 = BatchNormalization()(dense3)

    output = Dense(123, activation="sigmoid")(bn3)
    clf8 = Model(inputs=input1, outputs=output)
    clf8.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'] )

    
    clf1.fit(X[train], Y[train])
    clf2.fit(X[train], Y[train])
    clf3.fit(X[train], Y[train])
    clf7.fit(X[train], Y[train])
    clf8.fit(X[train], Y[train], batch_size=64, epochs=24,verbose=0 )
    
    pred_values1.append(clf1.predict(X[test]) )  
    pred_values2.append(clf2.predict(X[test]) )  
    pred_values3.append(clf3.predict(X[test]) )  
    pred_values7.append(clf7.predict(X[test]) ) 
    y_pred1 = clf8.predict(X[test])
    y_pred = np.zeros(y_pred1.shape)
    y_pred[y_pred1>0.22] = 1.0 
    pred_values8.append(y_pred)  
    
    GT_values.append(Y[test])


# TRAIN
a1 = np.concatenate(pred_values1, axis=0)
a2 = np.concatenate(pred_values2, axis=0)
a3 = np.concatenate(pred_values3, axis=0)
a7 = np.concatenate(pred_values7, axis=0)
a8 = np.concatenate(pred_values8, axis=0)

Y_meta=np.concatenate(GT_values, axis=0)

# VAL
a1_meta = clf1.predict(X_val)
a2_meta = clf2.predict(X_val)
a3_meta = clf3.predict(X_val)
a7_meta = clf7.predict(X_val)
y_pred1 = clf8.predict(X_val)
y_pred = np.zeros(y_pred1.shape)
y_pred[y_pred1>0.22] = 1.0 
a8_meta = y_pred

# TEST
t1 = clf1.predict(X2_TEST)
t2 = clf2.predict(X2_TEST)
t3 = clf3.predict(X2_TEST)
t7 = clf7.predict(X2_TEST)
y_pred1 = clf8.predict(X2_TEST)
y_pred = np.zeros(y_pred1.shape)
y_pred[y_pred1>0.22] = 1.0 
t8 = y_pred

1
2
3
4
5
6
7
8
9
10


In [53]:
stacked_input = np.stack((a1,a2,a3,a7,a8), axis=1)
stacked_val = np.stack((a1_meta, a2_meta,a3_meta,a7_meta,a8_meta), axis=1)
stacked_test = np.stack((t1,t2,t3,t7,t8), axis=1)
# train shape (7230, 3, 123)
# test shape (1808, 3, 123)

In [126]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

from keras.models import Model
from keras.layers import Input, Dense, SpatialDropout1D, Concatenate,BatchNormalization,Flatten,Dropout
from keras.callbacks import Callback
from sklearn.metrics import f1_score

models_num = 5
nodes = 123 * 8

finalscore = []
for itr in range(0,1):
    input1 = Input(shape=(models_num,123)  )
    dense1 = Dense(123, activation="relu")(input1)
    bn1 = BatchNormalization()(dense1)
    dr1 = Dropout(0.1)(bn1)

    #dense2 = Dense(123, activation="relu")(bn1)
    #bn2 = BatchNormalization()(dense2)
    #dr2 = Dropout(0.1)(bn2)
    #dense3 = Dense(123, activation="relu")(bn2)
    #bn3 = BatchNormalization()(dense3)

    flt= Flatten()(dr1)
    output = Dense(123, activation="sigmoid")(flt)
    meta_model =  Model(inputs=input1, outputs=output)

    meta_model.compile(loss='binary_crossentropy', 
                        optimizer='adam',
                        metrics=['accuracy'])

    meta_model.fit(x=stacked_input, y=Y_meta, batch_size=32, epochs=4,verbose=0 ) #,  callbacks=[early_stop,reduce_lr]

    y_pred1 = meta_model.predict(stacked_val, verbose=0)
    y_pred = np.zeros(y_pred1.shape)
    y_pred[y_pred1>0.3] = 1 
    scores = f1_score(Y_val, y_pred , average="micro")
    finalscore.append(scores)
    print("%s: %.3f%%" % ('f1 micro-average: ', scores))
    
# print("%.3f%% (+/- %.3f%%)" % (np.mean(finalscore), np.std(finalscore)))
# 0.3733333

f1 micro-average: : 0.373%


In [134]:
y_pred1 = meta_model.predict(stacked_val, verbose=0)
y_pred = np.zeros(y_pred1.shape)
y_pred[y_pred1>0.26] = 1 
scores = f1_score(Y_val, y_pred, average="micro")
print(scores)

0.38002960570582694


In [33]:
64 batch 4 epoch
_________________
0.34
0.334
0.33



x1 984 ---
0.33482267523038256
0.3122551762730834

x1 123
0.3441164006215567
0.3268612934767426




x1 123
x2 123
0.334876989869754
0.309953434225844
#0.32238135355237285  epoch 3   600
#0.3300403506330875   epoch 3   700

# WORK!
# need more than 0.3145503060743996

SyntaxError: invalid syntax (<ipython-input-33-1f6bb3b8e50a>, line 1)

In [ ]:
"""
import numpy as np
from pandas import DataFrame
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
seed = 2017
np.random.seed(seed)
from mlens.ensemble import SuperLearner
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier 
from sklearn.model_selection import train_test_split 
X, X_val, Y, Y_val = train_test_split(  X2_TRAIN, LABELS, test_size=0.2,random_state=52 , shuffle = True) #random_state=52 
from mlens.metrics import make_scorer
from sklearn.metrics import f1_score #, make_scorer
f1_scorer = make_scorer(f1_score, average="micro",greater_is_better=True)
ensemble = SuperLearner(scorer=f1_scorer, random_state=seed, verbose=2)
ensemble.add([OneVsRestClassifier(RandomForestClassifier(random_state=seed)), OneVsRestClassifier(SVC())])
ensemble.add_meta(OneVsRestClassifier(LogisticRegression()))
ensemble.fit(X, Y)
preds = ensemble.predict(X_val)
print("Fit data:\n%r" % ensemble.data)
print("Prediction score: %.3f" % accuracy_score(preds, Y_val))
"""

In [135]:
#  SAVE PREDICTIONS TO CSV

y_pred1 = meta_model.predict(stacked_test)
#y_pred1 = model_C.predict(X2_TEST)

y_pred = np.zeros(y_pred1.shape)
y_pred[y_pred1>0.26] = 1 

defectCodes = []
for each in label_onehoted:
    defectCodes.append(each)

predictions = []
column_num = len(y_pred[0])

for row in range(0, len(y_pred)):
    temp_list = []
    for column in range(0, column_num):
        if y_pred[row][column] == 1:
            temp_list.append(defectCodes[column])
            
    if 1 in temp_list:
        temp_list = []
        
    final_str = ''
    if len(temp_list) != 0:
        for e in temp_list:
            final_str += ','+str(e)
    else:
        final_str = ','

    predictions.append(final_str)    

    
from collections import OrderedDict
HEATID = x_test['HEATID'].values.tolist()
HEATID = list(OrderedDict.fromkeys(HEATID))

final_pred = []
for z in range(0, len(HEATID)):
    final_pred.append( str(HEATID[z]) + predictions[z] )

with open('predictions.csv','w') as file:
    for line in final_pred:
        file.write(line)
        file.write('\n')